# rotinas úteis

In [ ]:
omp_set_num_threads(int t)
omp_get_max_threads()
omp_get_num_threads()
omp_thread_num()
omp_get_num_procs()
omp_get_wtime()

In [ ]:
! time ./fibonacci]
//mede tempo do programa

In [ ]:
! cat /proc/cpuinfo
// ter informaçào da cpu

# diretivas

## parallel

In [ ]:
%%writefile a.c
#include <stdio.h>
#include <omp.h>

int main(){
    #pragma omp parallel
        printf("Hello from thread %d\n", omp_get_thread_num());
    return 0;
}

Writing a.c


In [ ]:
! gcc -fopenmp -o a a.c
! ./a

Hello from thread 1
Hello from thread 0


### private

In [ ]:
%%writefile b.c
#include <stdio.h>
#include <omp.h>

int main(){
    int a = -1;
    #pragma omp parallel private(a)
    {
        printf("dentro = %d\n", a);
        a = 123;
    }
    printf("fora = %d\n", a);
    return 0;
} // passa a variavel a para dentro de cada thread mas cada thread vai ser instancidada com um valor aleatorio, o que acontece aqui dentro não é passado pra fora
// por isso q printa for n ta +1

Writing b.c


In [ ]:
! gcc -fopenmp -o b b.c
! ./b

dentro = 0
dentro = 31611
fora = -1


### firstprivate

In [ ]:
%%writefile c.c
#include <stdio.h>
#include <omp.h>

int main(){
    int a = -1;
    #pragma omp parallel firstprivate(a)
    {
        printf("dentro = %d\n", a);
        a = 123;
    }
    printf("fora = %d\n", a);
    return 0;
}//bem parecida com a de cima, só que a variavel nao é inicializada com o valores aleatórios, ela é inicializada com o valor que ela jká possuia antes de começar o bloco paralelo

Writing c.c


In [ ]:
! gcc -fopenmp -o c c.c
! ./c

dentro = -1
dentro = -1
fora = -1


### shared

In [ ]:
%%writefile d.c
#include <stdio.h>
#include <omp.h>

int main(){
    int a = -1;
    #pragma omp parallel shared(a)
    {
        printf("dentro antes = %d\n", a);
        a = omp_get_thread_num();
        printf("dentro depois = %d\n", a);
    }
    printf("fora = %d\n", a);
    return 0;
} // você vai passar essa varivel para dentro do bloco paralelo, porém todas as threads vao ter acesso a essa variável no enderço de memória original della, ou seja, uma thread pode alterar o valro dessa var antes de outra
// e duas threads podem tentar acessar ela ao mesmo tempo o que pode gerar uma condição de corrida

Overwriting d.c


In [ ]:
! gcc -fopenmp -o d d.c
! ./d

dentro antes = -1
dentro depois = 1
dentro antes = 1
dentro depois = 0
fora = 0


### reduction

reduction vai fazer um loop


```
for i in sub_a:
    a (op)= i
```

onde i é o valor assumido por elem_sub_a ao final do bloco paralelo

In [ ]:
%%writefile e.c
#include <stdio.h>
#include <omp.h>

int main(){
    int a = 1;
    omp_set_num_threads(8);
    #pragma omp parallel reduction(*:a)
    {
        a = 2;
    }
    printf("fora = %d\n", a);
    return 0;
} //passa a var que nem o firstprivate, onde cada thread vai ter uma instância própria da var com a diferença de que o valor inicial em cada thread não é o valor que a var possuia antes do bloco paralelo,
// o valor que a var assume dentro de cada thread é 0, e ao final do bloco elas são unidas pela operação passada no reduction

Writing e.c


In [ ]:
! gcc -fopenmp -o e e.c
! ./e

fora = 256


## for

In [ ]:
%%writefile f.c
#include <stdio.h>
#include <omp.h>

#define SIZE 5


int main(){
    int a[SIZE];
    int b[SIZE];
    int c[SIZE];

    for (int i = 0; i < SIZE; i++) {
        a[i] = i + 1;
        b[i] = 2 * (i + 1);
    }

    for (int i = 0; i < SIZE; i++) {
            c[i] = a[i] + b[i];
        }

    printf("Vector a: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", a[i]);
    }
    printf("\n");

    printf("Vector b: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", b[i]);
    }
    printf("\n");

    printf("Vector c: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", c[i]);
    }
    printf("\n");

    return 0;
}

Overwriting f.c


In [ ]:
! gcc -fopenmp -o f f.c
! ./f

Vector a: 1 2 3 4 5 
Vector b: 2 4 6 8 10 
Vector c: 3 6 9 12 15 


In [ ]:
%%writefile g.c
#include <stdio.h>
#include <omp.h>

#define SIZE 5


int main(){
    int a[SIZE];
    int b[SIZE];
    int c[SIZE];

    for (int i = 0; i < SIZE; i++) {
        a[i] = i + 1;
        b[i] = 2 * (i + 1);
    }

    #pragma omp parallel for
        for (int i = 0; i < SIZE; i++) {
            c[i] = a[i] + b[i];
        }

    printf("Vector a: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", a[i]);
    }
    printf("\n");

    printf("Vector b: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", b[i]);
    }
    printf("\n");

    printf("Vector c: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", c[i]);
    }
    printf("\n");

    return 0;
}

Writing g.c


In [ ]:
! gcc -fopenmp -o g g.c
! ./g

Vector a: 1 2 3 4 5 
Vector b: 2 4 6 8 10 
Vector c: 3 6 9 12 15 


In [ ]:
%%writefile h.c
#include <stdio.h>
#include <omp.h>

#define SIZE 5


int main(){
    int a[SIZE];
    int b[SIZE];
    int c = 0;

    for (int i = 0; i < SIZE; i++) {
        a[i] = i + 1;
        b[i] = 2 * (i + 1);
    }

    for (int i = 0; i < SIZE; i++) {
            c += a[i] * b[i];
        }

    printf("Vector a: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", a[i]);
    }
    printf("\n");

    printf("Vector b: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", b[i]);
    }
    printf("\n");

    printf("sum c: %d\n", c);

    return 0;
}

Writing h.c


In [ ]:
! gcc -fopenmp -o h h.c
! ./h

Vector a: 1 2 3 4 5 
Vector b: 2 4 6 8 10 
sum c: 110


In [ ]:
%%writefile i.c
#include <stdio.h>
#include <omp.h>

#define SIZE 5


int main(){
    int a[SIZE];
    int b[SIZE];
    int c = 0;

    for (int i = 0; i < SIZE; i++) {
        a[i] = i + 1;
        b[i] = 2 * (i + 1);
    }

    #pragma omp parallel shared(a, b)
    #pragma omp for schedule(static) reduction(+:c)
        for (int i = 0; i < SIZE; i++) {
            c += a[i] * b[i];
        }

    printf("Vector a: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", a[i]);
    }
    printf("\n");

    printf("Vector b: ");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", b[i]);
    }
    printf("\n");

    printf("a dot b: %d\n", c);

    return 0;
}

Overwriting i.c


In [ ]:
! gcc -fopenmp -o i i.c
! ./i

Vector a: 1 2 3 4 5 
Vector b: 2 4 6 8 10 
a dot b: 110


In [ ]:
collapse(2)

## sections

In [ ]:
%%writefile j.c
#include <stdio.h>
#include <omp.h>

#define SIZE 5

void a(){
    printf("Hello from f_A\n");
}

void b(){
    printf("Hello from f_B\n");
}

void c(){
    printf("Hello from f_C\n");
}

int main(){

    #pragma omp parallel
    {
        #pragma omp sections
        {
            #pragma omp section
            a();
            #pragma omp section
            b();
            #pragma omp section
            c();
        }
    }

    return 0;
}// zona de sessões criadas onde  cada sessão vai ser executado por uma única thread

Overwriting j.c


In [ ]:
! gcc -fopenmp -o j j.c
! ./j

Hello from f_A
Hello from f_B
Hello from f_C




```
nowait
```



## sincronização

### single

In [ ]:
%%writefile k.c
#include <stdio.h>
#include <omp.h>

#define SIZE 5


int main(){

    #pragma omp parallel num_threads(8)
    {
        printf("Thread %d: iniciada\n", omp_get_thread_num());
        #pragma omp single
        printf("Total de threads = %d\n", omp_get_num_threads());
    }

    return 0;
} //n sei se é q nem o paralel ou q vc só pode colocar um for, o que tiver no single vai ser executado por uma única thread só

Overwriting k.c


In [ ]:
! gcc -fopenmp -o k k.c
! ./k

Thread 4: iniciada
Total de threads = 8
Thread 0: iniciada
Thread 3: iniciada
Thread 2: iniciada
Thread 6: iniciada
Thread 1: iniciada
Thread 7: iniciada
Thread 5: iniciada


### critical

In [ ]:
%%writefile l.c
#include <stdio.h>
#include <omp.h>

#define SIZE 5


int main(){
    int x = 0;
    #pragma omp parallel num_threads(8)
    {
        #pragma omp critical
        x++;
    }
    printf("x = %d\n", x);
    return 0;
} // o que tiver acontecendo dentro da zona crítica vai ser executado de maneira sequencial (que evita condição de corrida)

Overwriting l.c


In [ ]:
! gcc -fopenmp -o l l.c
! ./l

x = 8


### barrier

In [ ]:
%%writefile m.c
#include <stdio.h>
#include <omp.h>

void compute_a(int x){
    #pragma omp for
    for(int i=0; i<x; i++){
        printf("compute_a: i <=> %d\n", i);
    }
}

void compute_b(int x){
    #pragma omp for
    for(int i=0; i<x; i++){
        printf("compute_b: i <=> %d\n", i+100);
    }
}



int main(){
    int x = 5;
    #pragma omp parallel num_threads(8)
    {
        compute_a(x);
        //#pragma omp barrier
        compute_b(x);
    }
    return 0;
}

Overwriting m.c


In [ ]:
! gcc -fopenmp -o m m.c
! ./m

compute_a: i <=> 0
compute_a: i <=> 4
compute_a: i <=> 3
compute_a: i <=> 2
compute_a: i <=> 1
compute_b: i <=> 100
compute_b: i <=> 104
compute_b: i <=> 103
compute_b: i <=> 102
compute_b: i <=> 101


In [ ]:
%%writefile m.c
#include <stdio.h>
#include <omp.h>

void compute_a(int x){
    #pragma omp for
    for(int i=0; i<x; i++){
        printf("compute_a: i <=> %d\n", i);
    }
}

void compute_b(int x){
    #pragma omp for
    for(int i=0; i<x; i++){
        printf("compute_b: i <=> %d\n", i+100);
    }
}



int main(){
    int x = 5;
    #pragma omp parallel num_threads(8)
    {
        compute_a(x);
        #pragma omp barrier
        compute_b(x);
    }
    return 0;
}

Overwriting m.c


In [ ]:
! gcc -fopenmp -o m m.c
! ./m

compute_a: i <=> 0
compute_a: i <=> 2
compute_a: i <=> 4
compute_a: i <=> 3
compute_a: i <=> 1
compute_b: i <=> 100
compute_b: i <=> 102
compute_b: i <=> 104
compute_b: i <=> 103
compute_b: i <=> 101


# exercises

In [ ]:
%%writefile quest3.c
#include <stdio.h>
#include <omp.h>
#include <stdlib.h>



int main(){
    int N = 5;
    int *a, *b, *c;

    a = (int *)malloc(N * sizeof(int));
    b = (int *)malloc(N * sizeof(int));
    c = (int *)malloc(N * sizeof(int));

    #pragma omp parallel for
        for(int i =0; i < N; i++){
            a[i] = 2*i;
            b[i] = 2*i + 1;
            c[i] = 0;
        }

    #pragma omp parallel for
        for(int i =0; i < N; i++){
            c[i] = a[i] + b[i];
        }

    printf("Primeiro vetor: ");
    for (int i = 0; i < N; i++) {
        printf("%d ", a[i]);
    }
    printf("\n");

    printf("Segundo vetor: ");
    for (int i = 0; i < N; i++) {
        printf("%d ", b[i]);
    }
    printf("\n");


    printf("Terceiro vetor: ");
    for (int i = 0; i < N; i++) {
        printf("%d ", c[i]);
    }
    printf("\n");

    // Free memory allocated for vectors
    free(a);
    free(b);
    free(c);

    return 0;
}

In [ ]:
%%writefile ex1.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define SOMA_INICIAL 1000

int main(){
  int n = 25;
  int soma, a[n];

  for (int i=0; i<n; i++){
    a[i]=i;
  }

  soma = SOMA_INICIAL;

  #pragma omp parallel for reduction(+:soma)
  for (int i=0; i<n; i++){
    soma = a[i]+soma;
  }

  printf("Valor da soma: %d\n", soma);
  return 0;
}


Overwriting ex1.c


In [ ]:
! gcc -fopenmp -o ex1 ex1.c
! ./ex1

Valor da soma: 1300


In [ ]:
%%writefile ex2.c
#include <stdio.h>
#include <omp.h>

void parallelAddition (unsigned N, const double *A, const double *B, double *C)
{
    unsigned i;

    #pragma omp parallel for shared (A,B,C,N) private(i) schedule(static)
    for (i = 0; i < N; ++i)
    {
        C[i] = A[i] + B[i];
    }
}

int main (int argc, char *argv[]) {

    parallelAddition()

    return 0;
}

In [ ]:
! gcc -fopenmp -o ex1 ex1.c
! ./ex1

In [ ]:
%%writefile ex3.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <omp.h>

int main() {
    //const int tamanho = 1000;
    const int tamanho = 2<<18;
    int vetor1[tamanho];
    int vetor2[tamanho];
    int vetorSoma[tamanho];
    int i;

    // Inicializando a semente para geração de números aleatórios
    srand(time(NULL));

    double inicio, fim;
    inicio = omp_get_wtime();           //funcao omp de tempo

    // Inicializando os vetores com valores aleatórios entre 0 e 9
    for (i = 0; i < tamanho; i++) {
        vetor1[i] = rand() % 10;
        vetor2[i] = rand() % 10;
        vetorSoma[i] = 0;
    }

    // Calculando a soma dos elementos dos vetores e inserindo no vetorSoma
    for (i = 0; i < tamanho; i++) {
        vetorSoma[i] = vetor1[i] + vetor2[i];
    }

    // Calculando a soma de cada elemento de cada vetor
    int somaVetor1 = 0;
    int somaVetor2 = 0;
    int somaVetorSoma = 0;

    for (i = 0; i < tamanho; i++) {
        somaVetor1 += vetor1[i];
        somaVetor2 += vetor2[i];
        somaVetorSoma += vetorSoma[i];
    }

    fim = omp_get_wtime();           //funcao omp de tempo

    // Exibindo os resultados
    printf("Soma do vetor1: %d\n", somaVetor1);
    printf("Soma do vetor2: %d\n", somaVetor2);
    printf("Soma do vetor Soma: %d\n", somaVetorSoma);

    fim = omp_get_wtime();           //funcao omp de tempo
    printf("\n\nTempo total: %.6f segundos.\n", fim-inicio);

    return 0;
}


Writing ex3.c


In [ ]:
!gcc -o ex3 ex3.c -fopenmp
!./ex3

Soma do vetor1: 2357930
Soma do vetor2: 2359033
Soma do vetor Soma: 4716963


Tempo total: 0.027491 segundos.


In [ ]:
%%writefile fibonacci_tasks_inc.c

#include <stdio.h>
#include <omp.h>

long fib (int n) { return (n < 2 ? 1 : fib (n - 1) + fib (n - 2)); }

int main (int argc, char *argv[]) {
  int max = 40;
  #pragma omp parallel
    #pragma omp single
      for (int n = 1; n <= max; n++) /* bottom-up */
      //for (int n = max; n >= 1; n--) /* top-down */
        #pragma omp task
          printf ("%d: %d %ld\n", omp_get_thread_num(), n, fib (n));
  return 0;
}

In [ ]:
%env OMP_NUM_THREADS=4
%env OMP_SCHEDULE=static,1
! gcc -fopenmp -o fibonacci_tasks_inc fibonacci_tasks_inc.c
! time ./fibonacci_tasks_inc

In [ ]:
%%writefile quicksort.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <omp.h>

int compare (const char *str1, const char *str2) {
  int len1 = strlen (str1);
  int len2 = strlen (str2);
  if (len1 == len2) return strcmp (str1, str2);
  return len1 - len2;
}

void par_qsort (char **data, int lo, int hi,
                int (*compare)(const char *, const char*)) {
  if (lo > hi) return;
  int l = lo;
  int h = hi;
  char *p = data[(hi + lo) / 2];
  while (l <= h) {
    while (compare (data[l], p) < 0) l++;
    while (compare (data[h], p) > 0) h--;
    if (l <= h) {
      char *tmp = data[l]; data[l] = data[h]; data[h] = tmp;
      l++; h--;
    }
  }
  #pragma omp task final(h - lo < 1000)
    par_qsort (data, lo, h, compare);
  #pragma omp task final(hi - l < 1000)
    par_qsort (data, l, hi, compare);
}

int main (int argc, char *argv[]) {
  int seed = 0;
  int num_strings = 100;
  char **strings;
  srandom (seed);
  strings = (char**)malloc (num_strings * sizeof (char*));
  for (int s = 0; s < num_strings; s++) {
    int len = random() % 64;
    strings[s] = (char*)malloc ((len + 1) * sizeof (char));
    for (int c = 0; c < len; c++)
      strings[s][c] = 'A' + random() % 26;
    strings[s][len] = '\0';
  }
  #pragma omp parallel
    #pragma omp single
      par_qsort (strings, 0, num_strings - 1, compare);
  return 0;
}

In [ ]:
%env OMP_NUM_THREADS=4
%env OMP_SCHEDULE=static
! gcc -fopenmp -o quicksort quicksort.c
! ./quicksort

In [ ]:
%%writefile tempo.c

#include <stdio.h>
#include <omp.h>

int main (){
  double inicio, fim;
  inicio = omp_get_wtime();           //funcao omp de tempo
  if (omp_in_parallel())
    printf("Dentro da regiao paralela \n");
  else
    printf("Fora da regiao paralela \n");
  #pragma omp parallel num_threads(6)
    {
      int num_procs, max_threads, tid;
      tid = omp_get_thread_num();
      if (tid == 0) {
        num_procs = omp_get_num_procs();
        max_threads = omp_get_max_threads();
        printf("Numero de processadores disponiveis = %d \n", num_procs);
        printf("Numero maximo de threads = %d \n", max_threads);
        if (omp_in_parallel())
          printf("Dentro da regiao paralela\n");
        else
          printf("Fora da regiao paralela \n");
      }
    }
        fim = omp_get_wtime();      //funcao omp de tempo
        printf("Tempo gasto na execucao = %.6f segundos \n", fim-inicio);
}

In [ ]:
%env OMP_NUM_THREADS=4
%env OMP_SCHEDULE=static
! gcc -fopenmp -o tempo tempo.c
! ./tempo

In [ ]:
%%writefile ex1.c
#include <stdio.h>
#include <omp.h>

int main(){
    #pragma omp parallel num_threads(30)
    {
      #pragma omp single
      {
        printf("Impressao:\n");
      }
      if((omp_get_thread_num()%2)==0)
        printf(" %d \n", omp_get_thread_num());
    }
    printf("\n");
    return 0;
}

In [ ]:
! gcc -o ex1 ex1.c -fopenmp
! ./ex1

In [ ]:
%%writefile hugo.c
#include <stdio.h>
#include <omp.h>

int main(){
    int a = 0;
    #pragma omp parallel num_threads(5) reduction(+:a)
    {
        int x = omp_get_thread_num();
        printf("thread %d before assign: a === %d\n", x, a);
        a = x;
        printf("thread %d after assign: a === %d\n", x, a);
    }
    printf("a after all: %d\n", a);
    return 0;
}

Overwriting hugo.c


In [ ]:
! gcc -o hugo hugo.c -fopenmp
! ./hugo

thread 0 before assign: a === 0
thread 0 after assign: a === 0
thread 2 before assign: a === 0
thread 2 after assign: a === 2
thread 3 before assign: a === 0
thread 3 after assign: a === 3
thread 4 before assign: a === 0
thread 1 before assign: a === 0
thread 1 after assign: a === 1
thread 4 after assign: a === 4
a after all: 10


In [ ]:
%%writefile hugo2.c
#include <stdio.h>
#include <omp.h>

int main(){
    int N = 4;
    int sum = 0;
    #pragma omp parallel for num_threads(5) reduction(+:sum)
    for(int i = 0; i< N; i++){
        sum += 1;
    }
    printf("sum: %d\n", sum);
    return 0;
}

Overwriting hugo2.c


In [ ]:
! gcc -o hugo2 hugo2.c -fopenmp
! ./hugo2

sum: 4
